In [0]:
#3. 神经网络
#pytorch中 torch.nn 专门用于实现神经网络，其中nn.Module包含了网络层的搭建，以及一个方法 -- forward(input)，并返回网络的输出output
#实现一个经典的LeNet网络用来对字符进行分类

#重点！神经网络的标准训练流程 多个部分
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

  #这个init函数的作用只是定义输入输出
  def __init__(self):
    super(Net,self).__init__()
    #输入图像是单通道， 一层卷积核5x5大小，输出通道6
    self.conv1 = nn.Conv2d(1, 6, 5)#输入1，输出6channel，卷积核是5x5大小的
    #conv2，输入进6通道，输出16通道（也就是进行16次卷积操作），卷积核大小5x5
    self.conv2 = nn.Conv2d(6, 16, 5)
    #全连接层
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)#最后输出的是一个10通道的分类结果

  def forward(self,x):#前向传播整个的流程怎么走
    #max-pooling 采用一个2x2的滑动窗口
    x = F.max_pool2d(F.relu(self.conv1(x)),2)
    x = F.max_pool2d(F.relu(self.conv2(x)),2)
    x = x.view(-1,self.num_flat_features(x))#转换成一个向量？ 16 *5 *5的
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self,x): 
    #除了batch维度外的所有维度
    size = x.size()[1:]# 这里为什么要使用[1:],是因为pytorch只接受批输入，也就是说一次性输入好几张图片，那么输入数据张量的维度自然上升到了4维。【1:】让我们把注意力放在后3维上面
    num_features = 1
    for s in size:
      num_features *= s
    return num_features #16 * 5 * 5 ↑
net = Net()
print(net)
    
#这里必须实现forward函数
#在网络中，可以使用net.parameters()返回网络训练的参数（？）
params = list(net.parameters())
print("参数数量：",len(params))
# conv1.weight
print("第一个参数大小：", params[0].size())#这里可以一层一层的看

#以下代码简单测试这个网络
input = torch.randn(1,1,32,32)
out = net(input)
print("outtest:", out) #最终输出10个


#接着反向传播需要先清空梯度缓存，然后计算随机梯度进行反向传播
#print("show : ",net.zero_grad())
net.zero_grad()
#print("show2 : ",torch.randn(1,10))
out.backward(torch.randn(1,10))

#nn.Conv2d接受的输入是一个4维张量，nSamples * nChannels * Height * Width

In [11]:
#3.2损失函数loss func
#损失函数的输入是(output, target) 网络输出和真是标签对的数据，返回一个数值表示网络输出和真实标签的差距
#简单的损失函数是通过MSE均方误差来进行计算

output = net(input)
print("output:",output)
#定义伪标签,这里指的是这个target目标只是一个随便设置的，和网络输出无关，是一个randn
target = torch.rand(10)
print("target",target)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)#loss 也是一个1d的tensor

output: tensor([[-0.0270,  0.0323, -0.0253,  0.1146, -0.0260, -0.0662,  0.1373, -0.1628,
         -0.0450, -0.0090]], grad_fn=<AddmmBackward>)
target tensor([0.8017, 0.0283, 0.0251, 0.1488, 0.9078, 0.7367, 0.0570, 0.1321, 0.2928,
        0.7309])
tensor(0.2962, grad_fn=<MseLossBackward>)


In [12]:
#如果调用loss.backward() 那么整个图是可微分的，也就是说包括loss，图中的所有张量是变量，只要是requires_grad = True， .grad张量都会随着梯度一直累积
#MSELoss
print(loss.grad_fn)
#Linear Layer
print(loss.grad_fn.next_functions[0][0])
#Relu
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [13]:
#3.3反向传播
#反向传播只需要调用loss.backward()即可，首先要清空当前梯度缓存 .zero_grad()方法 否则造成梯度累加

#conv1层的偏置参数bias反向传播前后结果
net.zero_grad()
print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)




conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad before backward
tensor([ 0.0120, -0.0025,  0.0159, -0.0193, -0.0040,  0.0111])


In [0]:
#3.4更新权重
#采用随机梯度下降，方法更新权重规则 SGD 随机更新权重？ 按照这个公式对权重进行随机的更新
#weight = weight - learning_rate * gradient
#简单实现更新权重的更新例子
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

#为了采用更多的优化算法，不仅仅是SGD，还有其他的算法，为了采用不同的方法，采用torch.optim 库，例子如下


In [0]:
import torch.optim as optim
#创建优化器 【optimizer是更新权重用的！！！！！！！】
optimizer = optim.SGD(net.parameters(), lr = 0.01)

#在训练过程中执行下列操作
optimizer.zero_grad()#清空梯度缓存
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()